# Import Libraries

In [11]:
# %pip install --disable-pip-version-check \
#     torch==1.13.1 \
#     torchdata==0.5.1 --quiet

# %pip install \
#     transformers==4.27.2 \
#     datasets==2.11.0  --quiet

In [1]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer
from transformers import GenerationConfig
from sklearn.model_selection import train_test_split

C:\Users\Hanif\anaconda3\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import pandas as pd

# Load Dataset

In [3]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [3]:
df = pd.read_excel('result_fuzzy_jaccard.xlsx')

In [4]:
df.head(5)

,Product Name,Product SKU
0,Cantik,CANTIK CALCINIT
1,ZK,ZK Petro
2,za,ZA Petro
3,ZA@ 50kg,ZA Petro
4,UREA PIM NON SUBSIDI,Urea PIM


In [5]:
dataset = df[['Product Name','Product SKU']].dropna()

In [6]:
dataset.count()

Product Name    18490
Product SKU     18490
dtype: int64

In [7]:
import pandas as pd
from datasets import Dataset, DatasetDict

# Assuming you have loaded your pandas DataFrame into a variable 'df'
data = pd.DataFrame({
    'id': range(len(df)),
    'Product Name': df['Product Name'],
    'Product SKU': df['Product SKU']
})

data = data[data['Product Name'].notna()]
# Split the data into train, test, and validation sets
train_data, eval_test_data = train_test_split(data, test_size=0.3, random_state=42)
test_data, validation_data = train_test_split(eval_test_data, test_size=0.5, random_state=42)

# Convert each split into a Hugging Face Dataset
train_dataset = Dataset.from_pandas(train_data)
test_dataset = Dataset.from_pandas(test_data)
validation_dataset = Dataset.from_pandas(validation_data)

# Create a DatasetDict with these datasets
dataset = DatasetDict({
    'train': train_dataset,
    'test': test_dataset,
    'validation': validation_dataset
})

C:\Users\Hanif\anaconda3\lib\site-packages\pyarrow\pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


In [25]:
train_data.to_csv('train.csv')
test_data.to_csv('test.csv')
validation_data.to_csv('validation.csv')

In [8]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'Product Name', 'Product SKU', '__index_level_0__'],
        num_rows: 12943
    })
    test: Dataset({
        features: ['id', 'Product Name', 'Product SKU', '__index_level_0__'],
        num_rows: 2773
    })
    validation: Dataset({
        features: ['id', 'Product Name', 'Product SKU', '__index_level_0__'],
        num_rows: 2774
    })
})

example full and summary

In [10]:
example_indices = [40, 200]

dash_line = '-'.join('' for x in range(100))

for i, index in enumerate(example_indices):
    print(dash_line)
    print('Example ', i + 1)
    print(dash_line)
    print('Product POS:')
    print(dataset['test'][index]['Product Name'])
    print(dash_line)
    print('Product SKU:')
    print(dataset['test'][index]['Product SKU'])
    print(dash_line)
    print()

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
Product POS:
mascot 50ml
---------------------------------------------------------------------------------------------------
Product SKU:
Mascot
---------------------------------------------------------------------------------------------------

---------------------------------------------------------------------------------------------------
Example  2
---------------------------------------------------------------------------------------------------
Product POS:
npk pelangi 16-16-16 joss
---------------------------------------------------------------------------------------------------
Product SKU:
Pelangi 16-16-16
---------------------------------------------------------------------------------------------------



#

 # Model Development

In [22]:
# %pip install --upgrade pip
# %pip install --disable-pip-version-check \
#     torch==1.13.1 \
#     torchdata==0.5.1 --quiet

# %pip install \
#     transformers==4.27.2 \
#     datasets==2.11.0 \
#     evaluate==0.4.0 \
#     rouge_score==0.1.2 \
#     loralib==0.1.1 \
#     peft==0.3.0 --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 15.0 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 8.4 MB/s eta 0:00:00


In [11]:
model_name='google/flan-t5-base'

model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [12]:
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

## Without Prompt Engineering

In [13]:
for i, index in enumerate(example_indices):
    pos = dataset['test'][index]['Product Name']
    sku = dataset['test'][index]['Product SKU']

    inputs = tokenizer(pos, return_tensors='pt')
    output = tokenizer.decode(
        model.generate(
            inputs["input_ids"],
            max_new_tokens=50,
        )[0],
        skip_special_tokens=True
    )

    print(dash_line)
    print('Example ', i + 1)
    print(dash_line)
    print(f'INPUT PROMPT:\n{pos}')
    print(dash_line)
    print(f'BASELINE HUMAN SUMMARY:\n{sku}')
    print(dash_line)
    print(f'MODEL GENERATION - WITHOUT PROMPT ENGINEERING:\n{output}\n')

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
INPUT PROMPT:
mascot 50ml
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
Mascot
---------------------------------------------------------------------------------------------------
MODEL GENERATION - WITHOUT PROMPT ENGINEERING:
mascot 50ml

---------------------------------------------------------------------------------------------------
Example  2
---------------------------------------------------------------------------------------------------
INPUT PROMPT:
npk pelangi 16-16-16 joss
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
Pelangi 16-16-16
--------------------------------------------------------------------------------

## Zero shot interference with an instruct prompt

In [14]:
for i, index in enumerate(example_indices):
    pos = dataset['test'][index]['Product Name']
    sku = dataset['test'][index]['Product SKU']

    prompt = f"""
Give me the appropriate name for product SKU

{pos}

Summary:
    """

    # Input constructed prompt instead of the dialogue.
    inputs = tokenizer(prompt, return_tensors='pt')
    output = tokenizer.decode(
        model.generate(
            inputs["input_ids"],
            max_new_tokens=50,
        )[0],
        skip_special_tokens=True
    )

    print(dash_line)
    print('Example ', i + 1)
    print(dash_line)
    print(f'INPUT PROMPT:\n{prompt}')
    print(dash_line)
    print(f'BASELINE HUMAN SUMMARY:\n{sku}')
    print(dash_line)
    print(f'MODEL GENERATION - ZERO SHOT:\n{output}\n')

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
INPUT PROMPT:

Give me the appropriate name for product SKU

mascot 50ml

Summary:
    
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
Mascot
---------------------------------------------------------------------------------------------------
MODEL GENERATION - ZERO SHOT:
mascot 50ml

---------------------------------------------------------------------------------------------------
Example  2
---------------------------------------------------------------------------------------------------
INPUT PROMPT:

Give me the appropriate name for product SKU

npk pelangi 16-16-16 joss

Summary:
    
---------------------------------------------------------------------------------------------------
BASELINE HUMAN

In [56]:
test_data_indices = list(range(1, len(dataset['test'])))

In [58]:
import pandas as pd

# Assuming you have a DataFrame named 'dataset'
# with 'test' as one of its columns containing the examples
# and 'Product Name' and 'Product SKU' as the keys within each example

# Create an empty DataFrame to store the results
results_df = pd.DataFrame(columns=['Product Name', 'Product SKU', 'Model Output'])
list_pos = []
list_sku = []
list_predictions = []

# Loop through each example index
for i, index in enumerate(test_data_indices):
    pos = dataset['test'][index]['Product Name']
    sku = dataset['test'][index]['Product SKU']

    # First loop - Without Prompt Engineering
    inputs = tokenizer(pos, return_tensors='pt')
    output = tokenizer.decode(
        model.generate(
            inputs["input_ids"],
            max_new_tokens=50,
        )[0],
        skip_special_tokens=True
    )

    # Append results to DataFrame
    list_pos.append(pos)
    list_sku.append(sku)
    list_predictions.append(output)
#     results_df = results_df.append({'Product Name': pos, 'Product SKU': sku, 'Model Output': output}, ignore_index=True)

#     # Second loop - Zero Shot
    prompt = f"""
Product POS:

{pos}

predict product SKU?
"""

    inputs = tokenizer(prompt, return_tensors='pt')
    output = tokenizer.decode(
        model.generate(
            inputs["input_ids"],
            max_new_tokens=50,
        )[0],
        skip_special_tokens=True
    )

#     # Append results to DataFrame
#     results_df = results_df.append({'Product Name': pos, 'Product SKU': sku, 'Model Output': output}, ignore_index=True)

# # Display the results DataFrame
# print(results_df)

In [59]:
df_without_prompt = pd.DataFrame({'Product Name':list_pos,'Product SKU':list_sku,'Predict SKU':list_predictions})

In [64]:
df_without_prompt.to_csv('df_without_prompt.csv')

In [60]:
df_without_prompt

,Product Name,Product SKU,Predict SKU
0,Sidalaku 1L,Sidalaku,Sidalaku 1L
1,Tabas@200 ml,Tabas,Tabas@200 ml
2,regent 50,Regent,regent 50
3,NPK PUSRI,NPK Pusri 15-15-15,NPK PUSRI
4,Agil,Agil,"Agil, the king of the sultanate, is a sultan o..."
...,...,...,...
2767,Centamec 100ml,Centamec,Centamec 100ml
2768,pegasus 80ml,Pegasus,pegasus 80ml
2769,Mip cinta@50gr,Mipcinta,Mip cinta@50gr
2770,NPK PUSRI 15.15.15,NPK Pusri 15-15-15,NPK PUSRI 15.15.15


## Zero shot with template instruct prompt from FLAN T-5 template prompt

In [15]:
for i, index in enumerate(example_indices):
    pos = dataset['test'][index]['Product Name']
    sku = dataset['test'][index]['Product SKU']

    prompt = f"""
Product POS:

{pos}

predict product SKU?
"""

    inputs = tokenizer(prompt, return_tensors='pt')
    output = tokenizer.decode(
        model.generate(
            inputs["input_ids"],
            max_new_tokens=50,
        )[0],
        skip_special_tokens=True
    )

    print(dash_line)
    print('Example ', i + 1)
    print(dash_line)
    print(f'INPUT PROMPT:\n{prompt}')
    print(dash_line)
    print(f'BASELINE HUMAN SUMMARY:\n{sku}\n')
    print(dash_line)
    print(f'MODEL GENERATION - ZERO SHOT:\n{output}\n')

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
INPUT PROMPT:

Product POS:

mascot 50ml

predict product SKU?

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
Mascot

---------------------------------------------------------------------------------------------------
MODEL GENERATION - ZERO SHOT:
tetrasodium phosphate

---------------------------------------------------------------------------------------------------
Example  2
---------------------------------------------------------------------------------------------------
INPUT PROMPT:

Product POS:

npk pelangi 16-16-16 joss

predict product SKU?

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
Pelangi 16-16-16

---------

In [61]:
# test_data_indices = list(range(1,10))
test_data_indices = list(range(1, len(dataset['test'])))
list_pos = []
list_sku = []
list_predictions = []

for i, index in enumerate(test_data_indices):
    pos = dataset['test'][index]['Product Name']
    sku = dataset['test'][index]['Product SKU']

    prompt = f"""
Product POS:

{pos}

Give me the appropriate name for product SKU
"""

    inputs = tokenizer(prompt, return_tensors='pt')
    output = tokenizer.decode(
        model.generate(
            inputs["input_ids"],
            max_new_tokens=50,
        )[0],
        skip_special_tokens=True
    )
    # Append results to DataFrame
    list_pos.append(pos)
    list_sku.append(sku)
    list_predictions.append(output)
df_prompt_engineer = pd.DataFrame({'Product Name':list_pos,'Product SKU':list_sku,'Predict SKU':list_predictions})

In [62]:
df_prompt_engineer

,Product Name,Product SKU,Predict SKU
0,Sidalaku 1L,Sidalaku,Sidalaku
1,Tabas@200 ml,Tabas,Tabas@200
2,regent 50,Regent,Regent
3,NPK PUSRI,NPK Pusri 15-15-15,NPK
4,Agil,Agil,Agil
...,...,...,...
2767,Centamec 100ml,Centamec,Centamec
2768,pegasus 80ml,Pegasus,Pegasus
2769,Mip cinta@50gr,Mipcinta,cinta@50gr
2770,NPK PUSRI 15.15.15,NPK Pusri 15-15-15,NPK PUSRI


In [63]:
df_prompt_engineer.to_csv('df_prompt_engineer.csv')

### One shot interference

In [65]:
def make_prompt(example_indices_full, example_index_to_summarize):
    prompt = ''
    for index in example_indices_full:
        pos = dataset['test'][index]['Product Name']
        sku = dataset['test'][index]['Product SKU']

        # The stop sequence '{summary}\n\n\n' is important for FLAN-T5. Other models may have their own preferred stop sequence.
        prompt += f"""
Raw name:

{pos}

Give me the appropriate name for product SKU?
{sku}


"""

    pos = dataset['test'][example_index_to_summarize]['Product Name']

    prompt += f"""
Raw name:

{pos}

Give me the appropriate name for product SKU?
"""

    return prompt

In [66]:
example_indices_full = [40]
example_index_to_summarize = 200

one_shot_prompt = make_prompt(example_indices_full, example_index_to_summarize)

print(one_shot_prompt)


Raw name:

mascot 50ml

Give me the appropriate name for product SKU?
Mascot



Raw name:

npk pelangi 16-16-16 joss

Give me the appropriate name for product SKU?



In [68]:
pos = dataset['test'][example_index_to_summarize]['Product Name']

inputs = tokenizer(one_shot_prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        max_new_tokens=50,
    )[0],
    skip_special_tokens=True
)

print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{pos}\n')
print(dash_line)
print(f'MODEL GENERATION - ONE SHOT:\n{output}')

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
npk pelangi 16-16-16 joss

---------------------------------------------------------------------------------------------------
MODEL GENERATION - ONE SHOT:
npk pelangi


In [71]:
# test_data_indices = list(range(1, len(dataset['test'])))
test_data_indices = list(range(1, len(dataset['test'])))
for i, index in enumerate(test_data_indices):
    pos = dataset['test'][index]['Product Name']
    sku = dataset['test'][index]['Product SKU']
    inputs = tokenizer(one_shot_prompt, return_tensors='pt')
    output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        max_new_tokens=50,
    )[0],
    skip_special_tokens=True
)
# Append results to DataFrame
    list_pos.append(pos)
    list_sku.append(sku)
    list_predictions.append(output)
df_one_shot = pd.DataFrame({'Product Name':list_pos,'Product SKU':list_sku,'Predict SKU':list_predictions})

In [72]:
df_one_shot.to_csv('df_one_shot.csv')

In [73]:
df_one_shot

,Product Name,Product SKU,Predict SKU
0,Sidalaku 1L,Sidalaku,Sidalaku
1,Tabas@200 ml,Tabas,Tabas@200
2,regent 50,Regent,Regent
3,NPK PUSRI,NPK Pusri 15-15-15,NPK
4,Agil,Agil,Agil
...,...,...,...
5540,Centamec 100ml,Centamec,npk pelangi
5541,pegasus 80ml,Pegasus,npk pelangi
5542,Mip cinta@50gr,Mipcinta,npk pelangi
5543,NPK PUSRI 15.15.15,NPK Pusri 15-15-15,npk pelangi


## A few shot interference

In [74]:
example_indices_full = [40, 80, 120]
example_index_to_summarize = 200

few_shot_prompt = make_prompt(example_indices_full, example_index_to_summarize)

print(few_shot_prompt)


Raw name:

mascot 50ml

Give me the appropriate name for product SKU?
Mascot


Raw name:

Ecofert 1kg

Give me the appropriate name for product SKU?
Ecofert


Raw name:

spuyer

Give me the appropriate name for product SKU?
Spuyer


Raw name:

npk pelangi 16-16-16 joss

Give me the appropriate name for product SKU?



In [32]:
sku = dataset['test'][example_index_to_summarize]['Product SKU']

inputs = tokenizer(few_shot_prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        max_new_tokens=50,
    )[0],
    skip_special_tokens=True
)

print(pos)
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{sku}\n')
print(dash_line)
print(f'MODEL GENERATION - FEW SHOT:\n{output}')

npk pelangi 16-16-16 joss
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
Pelangi 16-16-16

---------------------------------------------------------------------------------------------------
MODEL GENERATION - FEW SHOT:
npk pelangi


In [75]:
# test_data_indices = list(range(1, len(dataset['test'])))
test_data_indices = list(range(1, len(dataset['test'])))
for i, index in enumerate(test_data_indices):
    pos = dataset['test'][index]['Product Name']
    sku = dataset['test'][index]['Product SKU']
    inputs = tokenizer(few_shot_prompt, return_tensors='pt')
    output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        max_new_tokens=50,
    )[0],
    skip_special_tokens=True
)
# Append results to DataFrame
    list_pos.append(pos)
    list_sku.append(sku)
    list_predictions.append(output)
df_few_shot = pd.DataFrame({'Product Name':list_pos,'Product SKU':list_sku,'Predict SKU':list_predictions})

In [76]:
df_few_shot

,Product Name,Product SKU,Predict SKU
0,Sidalaku 1L,Sidalaku,Sidalaku
1,Tabas@200 ml,Tabas,Tabas@200
2,regent 50,Regent,Regent
3,NPK PUSRI,NPK Pusri 15-15-15,NPK
4,Agil,Agil,Agil
...,...,...,...
8312,Centamec 100ml,Centamec,npk pelangi
8313,pegasus 80ml,Pegasus,npk pelangi
8314,Mip cinta@50gr,Mipcinta,npk pelangi
8315,NPK PUSRI 15.15.15,NPK Pusri 15-15-15,npk pelangi


In [77]:
df_few_shot.to_csv('df_few_shot.csv')

## General Configuration Parameter for Interfence

In [33]:
#generation_config = GenerationConfig(max_new_tokens=50)
#generation_config = GenerationConfig(max_new_tokens=10)
#generation_config = GenerationConfig(max_new_tokens=50, do_sample=True, temperature=0.1)
#generation_config = GenerationConfig(max_new_tokens=50, do_sample=True, temperature=0.5)
generation_config = GenerationConfig(max_new_tokens=50, do_sample=True, temperature=1.0)

inputs = tokenizer(few_shot_prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        generation_config=generation_config,
    )[0],
    skip_special_tokens=True
)
print(dash_line)
print(f'dialogue:\n{pos}')
print(dash_line)
print(f'MODEL GENERATION - FEW SHOT:\n{output}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{sku}\n')

---------------------------------------------------------------------------------------------------
dialogue:
npk pelangi 16-16-16 joss
---------------------------------------------------------------------------------------------------
MODEL GENERATION - FEW SHOT:
npk pelangi
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
Pelangi 16-16-16



# Fine Tuning Model

In [9]:
import time
import torch
from transformers import Trainer, TrainingArguments

if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [10]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer
import torch
import time
import evaluate
import pandas as pd
import numpy as np

In [11]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [12]:
model_name='google/flan-t5-base'

original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [13]:
original_model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo):

In [14]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

print(print_number_of_trainable_model_parameters(original_model))

trainable model parameters: 247577856
all model parameters: 247577856
percentage of trainable model parameters: 100.00%


In [15]:
def tokenize_function(example):
    start_prompt = 'Give me the appropriate name for product SKU.\n\n'
    end_prompt = '\n\Product SKU: '

    # Check if "dialogue" is not None
    if example["Product Name"] is not None:
        prompt = [start_prompt + dialogue + end_prompt for dialogue in example["Product SKU"]]
        example['input_ids'] = tokenizer(prompt, padding="max_length", truncation=True, return_tensors="pt").input_ids
    else:
        # Handle the case where "dialogue" is None
        example['input_ids'] = [[]]  # Provide an empty list if "dialogue" is None

    example['labels'] = tokenizer(example["Product SKU"], padding="max_length", truncation=True, return_tensors="pt").input_ids

    return example

# The dataset actually contains 3 different splits: train, validation, test.
# The tokenize_function code is handling all data across all splits in batches.
tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(['id', 'Product Name', 'Product SKU'])

In [16]:
print(f"Shapes of the datasets:")
print(f"Training: {tokenized_datasets['train'].shape}")
print(f"Validation: {tokenized_datasets['validation'].shape}")
print(f"Test: {tokenized_datasets['test'].shape}")

print(tokenized_datasets)

Shapes of the datasets:
Training: (12943, 3)
Validation: (2774, 3)
Test: (2773, 3)
DatasetDict({
    train: Dataset({
        features: ['__index_level_0__', 'input_ids', 'labels'],
        num_rows: 12943
    })
    test: Dataset({
        features: ['__index_level_0__', 'input_ids', 'labels'],
        num_rows: 2773
    })
    validation: Dataset({
        features: ['__index_level_0__', 'input_ids', 'labels'],
        num_rows: 2774
    })
})


## Full Fine Tuning

In [24]:
torch.cuda.empty_cache()

In [22]:
output_dir = './'
batch_size = 8
training_args = TrainingArguments(
    output_dir=output_dir,
    learning_rate=1e-5,
    num_train_epochs=1,
    weight_decay=0.01,
    logging_steps=1,
    max_steps=1,
    per_device_train_batch_size=batch_size,  # Set the batch size here
    per_device_eval_batch_size=batch_size,   # Set the batch size for evaluation here
)

trainer = Trainer(
    model=original_model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation']
)
trainer.train()

OutOfMemoryError: CUDA out of memory. Tried to allocate 12.00 MiB (GPU 0; 6.00 GiB total capacity; 11.80 GiB already allocated; 0 bytes free; 11.87 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

# Evaluation

In [36]:
df_few_shot = pd.read_csv('df_few_shot.csv')
df_one_shot = pd.read_csv('df_one_shot.csv')
df_prompt_engineer = pd.read_csv('df_prompt_engineer.csv')
df_without_prompt = pd.read_csv('df_without_prompt.csv')

In [37]:
import pandas as pd
from sklearn.metrics import accuracy_score

# Assuming df_few_shot, df_one_shot, df_prompt_engineer, df_without_prompt are your DataFrames

# Create a list of tuples with DataFrame names and corresponding DataFrames
dataframes = [
    ('FLAN T-5 Few Shot', df_few_shot),
    ('FLAN T-5 One Shot', df_one_shot),
    ('FLAN T-5 Prompt Engineer', df_prompt_engineer),
    ('FLAN T-5 Without Prompt Engineer', df_without_prompt),
]

# Initialize an empty list to store results
results = []

# Loop through the dataframes
for name, df in dataframes:
    # Assuming 'Product SKU' and 'Predict SKU' are the column names in your DataFrames
    y_true = df['Product SKU'].astype(str).tolist()
    y_pred = df['Predict SKU'].astype(str).tolist()

    # Calculate accuracy
    accuracy = accuracy_score(y_true, y_pred)*100

    # Append results to the list
    results.append((name, accuracy))

# Create a summary DataFrame
summary_df = pd.DataFrame(results, columns=['Model', 'Accuracy(%)'])

In [38]:
summary_df

,Model,Accuracy(%)
0,FLAN T-5 Few Shot,11.254058
1,FLAN T-5 One Shot,16.880072
2,FLAN T-5 Prompt Engineer,33.766234
3,FLAN T-5 Without Prompt Engineer,2.561328


# Predict All of Data

In [40]:
fuzzy_jaccard = pd.read_excel('result_fuzzy_jaccard.xlsx')

In [41]:
fuzzy_jaccard

,Product Name,Product SKU
0,Cantik,CANTIK CALCINIT
1,ZK,ZK Petro
2,za,ZA Petro
3,ZA@ 50kg,ZA Petro
4,UREA PIM NON SUBSIDI,Urea PIM
...,...,...
18494,SPRAYER CBA,Hndsprayercba
18495,SARNET,Sarento
18496,PROMOTOP 20 L,Promotop
18497,BABLAS 20 L,Bablas


In [43]:
total_pos = pd.read_excel('Product Name from PoS Transactions.xlsx')

In [44]:
total_pos

,Product Name
0,Pupuk Urea N 46%
1,Pupuk Amonium Sulfat ZA
2,Pupuk Super Fosfat SP-36
3,Pupuk NPK Phonska
4,Pupuk NPK Formula Khusus
...,...
43997,Extra one 680 EC @ 250 ml
43998,Extra One 680 SC @ 500 ml
43999,JARING ARWANA @ 100 METER
44000,Terong Puma F1 @ 5 gram


In [45]:
pos_fuzzy = fuzzy_jaccard['Product Name']

In [57]:
total_pos['Product Name'] = total_pos['Product Name'].str.lower()
fuzzy_data = pos_fuzzy.str.lower()
df_difference = total_pos[~total_pos['Product Name'].isin(fuzzy_data)]

In [60]:
predict_dataset = Dataset.from_pandas(df_difference)

C:\Users\Hanif\anaconda3\lib\site-packages\pyarrow\pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


In [61]:
predict_data = DatasetDict({
    'predict': predict_dataset
})

In [33]:
model = AutoModelForSeq2SeqLM.from_pretrained("my_model")
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [63]:
# test_data_indices = list(range(1,10))
test_data_indices = list(range(1, len(predict_data['predict'])))
list_pos = []
list_predictions = []

for i, index in enumerate(test_data_indices):
    pos = predict_data['predict'][index]['Product Name']

    prompt = f"""
Product POS:

{pos}

Give me the appropriate name for product SKU
"""

    inputs = tokenizer(prompt, return_tensors='pt')
    output = tokenizer.decode(
        model.generate(
            inputs["input_ids"],
            max_new_tokens=50,
        )[0],
        skip_special_tokens=True
    )
    # Append results to DataFrame
    list_pos.append(pos)
    list_predictions.append(output)
df_predict = pd.DataFrame({'Product Name':list_pos,'Predict SKU':list_predictions})

In [74]:
df_predict = df_predict.rename(columns={'Predict SKU':'Product SKU'})

In [75]:
df_full = pd.concat([df_predict,fuzzy_jaccard])

In [76]:
df_full = df_full.drop_duplicates()

In [78]:
df_full.to_csv('df_full.csv')

In [79]:
df_predict

,Product Name,Product SKU
0,pupuk super fosfat sp-36,Pupuk
1,pupuk npk phonska,Pupuk
2,pupuk npk formula khusus,Pupuk
3,pupuk organik granul,Pupuk
4,pupuk organik cair,Pupuk
...,...,...
25535,extra one 680 ec @ 250 ml,ec
25536,extra one 680 sc @ 500 ml,epsom salts
25537,jaring arwana @ 100 meter,jaring arwana
25538,terong puma f1 @ 5 gram,terong puma


In [98]:
new_product_sku = pd.read_excel('new_product_sku.xlsx')
result = pd.read_excel('result.xlsx')
catalog = pd.read_excel('Product Catalog.xlsx')
pos = pd.read_excel('Product Name from PoS Transactions.xlsx')

In [99]:
catalog['cleaned_name'] = catalog['Product SKU'].str.lower().str.replace('-', ' ').str.replace(',', ' ').str.replace(r'@.*', '', regex=True)
pos['cleaned_name'] = pos['Product Name'].str.lower().str.replace('-', ' ').str.replace(',', ' ').str.replace(r'@.*', '', regex=True)

In [100]:
pos['cleaned_name'] = pos['cleaned_name'].astype(str)
catalog['cleaned_name'] = catalog['cleaned_name'].astype(str)

In [101]:
pos['cleaned_name'] = pos['cleaned_name'].astype(str)
catalog['cleaned_name'] = catalog['cleaned_name'].astype(str)

In [82]:
new_product_sku['New Product SKU'] = new_product_sku['New Product SKU'].str.title()

In [83]:
new_product_sku = new_product_sku.dropna()

In [84]:
new_product_sku = new_product_sku.rename(columns={'New Product SKU':'Product SKU'})
new_product_sku = new_product_sku[['Product Name','Product SKU']]
result = pd.concat([result,new_product_sku])

In [85]:
new_product_sku = new_product_sku.rename(columns={'New Product SKU':'Product SKU'})
new_product_sku = new_product_sku[['Product Name','Product SKU']]
result = pd.concat([result,new_product_sku])

In [86]:
result = result.drop_duplicates()

In [89]:
new_catalog = pd.DataFrame({'Product SKU':result['Product SKU'].unique(),'Brand':'','Type':'','Formula':''})

In [91]:
new_catalog['cleaned_name'] = new_catalog['Product SKU'].str.lower().str.replace('-', ' ').str.replace(',', ' ').str.replace(r'@.*', '', regex=True)
new_catalog['cleaned_name'] = new_catalog['cleaned_name'].astype(str)

In [102]:
update_catalog = pd.concat([catalog,new_catalog])

In [103]:
update_catalog

,Product SKU,Brand,Type,Formula,cleaned_name
0,Urea Petro,PIHC,Urea,NaN,urea petro
1,Urea PIM,PIHC,Urea,NaN,urea pim
2,Urea Nitrea,PIHC,Urea,NaN,urea nitrea
3,Urea Daun Buah,PIHC,Urea,NaN,urea daun buah
4,Urea Pusri,PIHC,Urea,NaN,urea pusri
...,...,...,...,...,...
4460,Goodfer,,,,goodfer
4461,Greenbig,,,,greenbig
4462,Gramoxaon,,,,gramoxaon
4463,Propel,,,,propel


In [104]:
new_catalog_1 = pd.DataFrame({'Product SKU':df_predict['Product SKU'].unique(),'Brand':'','Type':'','Formula':''})

In [105]:
new_catalog_1['cleaned_name'] = new_catalog_1['Product SKU'].str.lower().str.replace('-', ' ').str.replace(',', ' ').str.replace(r'@.*', '', regex=True)
new_catalog_1['cleaned_name'] = new_catalog_1['cleaned_name'].astype(str)

In [106]:
update_catalog = pd.concat([update_catalog,new_catalog_1])

In [108]:
update_catalog.to_csv('Update_Catalog.csv')

In [121]:
update_catalog = update_catalog[update_catalog['Product SKU']!='Pupuk']

In [113]:
df_full = df_full[df_full['Product SKU'].str.match(r'^\D')]

In [116]:
df_pupuk = df_full[df_full['Product SKU']=='Pupuk']

In [123]:
def calculate_jaccard_similarity(a, b):
    a_set = set(a.split())
    b_set = set(b.split())
    return len(a_set.intersection(b_set)) / len(a_set.union(b_set))

In [ ]:
# Create an empty list to store the results
result_data = []

for pos_name in pos['cleaned_name']:
    best_similarity = 0
    best_catalog_name = ''

    for catalog_name in catalog['cleaned_name']:
        similarity = calculate_jaccard_similarity(pos_name, catalog_name)
        if similarity > best_similarity:
            best_similarity = similarity
            best_catalog_name = catalog_name

    result_data.append([pos_name, best_catalog_name, best_similarity])

# Create the result DataFrame from the result_data list
result_df = pd.DataFrame(result_data, columns=['pos_cleaned_name', 'best_catalog_name', 'jaccard_similarity_score'])